This notebook is the first main step of our methodology. Here we have a pre-existing file (PlateBoundaryTypes) which is a summary of the plate boundaries in the plate model (e.g. lat/lon of start/end points for each segment of a ridge/trench/fault, divergence speed, length, obliquity etc.). We use that data, alongside estimates in the literature of crustal thickness, subdivision of crustal lithologies (e.g. upper/lower volcanics, dykes, gabbros etc.), a parameterisation of serpentinisation etc. to the build the model for carbon storage and serpentinite in ocean crust at ridges, at each time step. In order to properly sample all the parameter space, we do each calcualtion 10000 times per ridge segment. We then save the mean of these values into a big dictionary, which we will access in the next notebook as we start rotating the points through time. We take just the mean because we have the full distribution already saved (3_Uncertainty_Distributions_per_spreading_rate.ipnyb), and we will use that distribution to perturb our result at a trench.

In the corresponding paper (Merdith et al. 2019) we wanted to explore the uncertainy inherent in spreading rate for ocean basins that have no recorded spreading history (i.e. prior to the Jurassic mostly). We differentiated our two approahces as (i) plate model spreading rate (PMSR) and (ii) Pacific Ocean spreading rate (POSR).

(i) uses the spreading rate in the plate model as it as been constructed 
(ii) we assume that the spreading rate for the Pacific Ocean since 83 Ma is representative of 'external' oceans (ie. oceans without continental lithosphere and fully ringed by subduction). We built a distribution of expected Pacific Ocean spreading rates (see Notebook 1_Spreading_rate_distributions.ipynb) and instead of using the spreading rate for external oceans in times prior to the Jurassic, we randomly select a spreading rate from that distribution.  As is, these lines are currently blocked out in the below code.

References

Merdith, A.S., Atkins, S.E., and Tetley, M.G. (2019). Tectonic controls on carbon and serpentinite storage in subducted upper oceanic lithosphere for the past 320 Ma. Frontiers: Earth Science.

In [3]:
import numpy as np
import pandas as pd
import MOR_crust_calculation
import pickle
from scipy import interpolate
from collections import defaultdict
import MOR_characterisation_serp_flux_FINAL
from datetime import datetime
import pickle
import dill

In [2]:
#select random pacific spreading rate
def random_spreading(samples,distribution):
    y = np.random.randint(1001,size=samples)
    return distribution[y]

In [11]:
def crust_characterisation(start, stop, step, df):

    '''
    Characterises the crust at each time step using a plate model.
    '''
    C_storage = defaultdict(lambda: defaultdict(list))

    # Set up time array
    min_time = start
    max_time = stop
    time_step = step
    times = np.arange(min_time,max_time + time_step,time_step)

    # deviation angle (in degrees) to split transforms vs. ridges from
    deviation_angle= 70
    #time resolution to extract data
    time_resolution = 1
    SR = 'Spreading Rate'
    prop_per = 'Peridotite'
    DS = 'Degree of Serpentinisation'
    CO2wt_mean = 'CO2 mean'
    CO2wt_std = 'CO2 std'
    thick_mean = 'Thickness Mean'
    thick_std = 'Thickness std'
    vertical_area_mean = 'Vertical Area Mean'
    vertical_area_std = 'Vertical Area std'
    C_total = 'C-prod'
    C_total_vertical = 'C-prod_vertical'
    C_volcanic_serp = 'C-Volcanic_serp'
    serpentinites_thick = 'Serpentinites_raw'
    serpentinites_vertical = 'Serpentinites_raw_vertical'
    PDF_params = 'PDF Parameters'
    length = 'Boundary-Length'
    boundary = 'Boundary-Type'
    StartLat = 'Start-Lat-Point'
    StartLon = 'Start-Lon-Point'
    EndLat = 'End-Lat-Point'
    EndLon = 'End-Lon-Point'
    RightPlateID = 'RightPlate'
    LeftPlateID = 'LeftPlate'
    Index = 'Index'

    panthalassa_plate_ids = ['902',
                                       '919',
                                       '926']

    #inititate loop to extract data
    for time in times:
        print time,  'Ma'
        #we only need to cut up spreading ridges by velocity
        subset1 = df[(df['Time_Ma']>=time)
                  & (df['Time_Ma']<(time+time_resolution))
                  & (df['FeatureType']=='gpml:MidOceanRidge')
                  & (np.abs(df['Deviation_mod_deg'])<=deviation_angle)]
        #print len(subset1)

        #create temporary storage for values
        C_storage[time][SR] = []
        C_storage[time][prop_per] = []
        C_storage[time][DS] = []
        C_storage[time][CO2wt_mean] = []
        C_storage[time][CO2wt_std] = []
        C_storage[time][thick_mean] = []
        C_storage[time][thick_std] = []
        C_storage[time][vertical_area_mean] = []
        C_storage[time][vertical_area_std] = []
        C_storage[time][serpentinites_thick] = []
        C_storage[time][serpentinites_vertical] = []
        C_storage[time][C_volcanic_serp] = []
        C_storage[time][C_total] = []
        C_storage[time][C_total_vertical] = []
        C_storage[time][PDF_params] = []
        C_storage[time][length] = []
        C_storage[time][boundary] = []
        C_storage[time][StartLat] = []
        C_storage[time][StartLon] = []
        C_storage[time][EndLat] = []
        C_storage[time][EndLon] = []
        C_storage[time][RightPlateID] = []
        C_storage[time][LeftPlateID] = []
        C_storage[time][Index] = []

        for index, row in subset1.iterrows():


           #returns full spreading rate/velocity in cm/year, times by 10 to convert to km/Ma
            velocity = np.ones(samples)
            velocity = velocity*row.Plate_Velocity*10

            #block out if not using plate model
            #for POSR
            #if time > 180:
            #    new_conjugate_plates = (row.RightPlate,row.LeftPlate)
            #    if all(x in panthalassa_plate_ids for x in new_conjugate_plates) == True:
            #        velocity = np.ones(samples)
            #        velocity = velocity*random_spreading(samples,spread_dist)*10
            #        #print 'here', velocity

            #calculate the mean bottom water temperature for the first 20 Ma existence of a parcel of ocean crust
            temp_tmp = []
            bottom_water_temperature = np.ones(samples)
            for i in np.arange(time,time-21,-1):
                if i < 0:
                    break
                temp_tmp.append(bottom_water_temperature_curve[i])
            bottom_water_temperature = bottom_water_temperature * np.mean(temp_tmp)

            per = MOR_characterisation_serp_flux_FINAL.SR_and_peridotite(samples,
                                                                         velocity)

            volcanic_percent = 100 - per
            asymmetry_factor = 1
            calc_length = row.Length_km

            #calculate variables for mid ocean ridge segments

            calc_length = row.Length_km
            width = velocity
            #as all velocitie are the same for each point, we just check the first
            if velocity[0] <= 40:
                asymmetry_factor = 1
                thickness = MOR_characterisation_serp_flux_FINAL.SR_and_thickness_slow_ultraslow(samples)

                tmp_DS = MOR_characterisation_serp_flux_FINAL.SR_and_dsSurf_slow_ultraslow(samples,
                                                                                           velocity,
                                                                                          thickness) #DS and thickness is called within this function
                #print DS
                carbon_max, CO2_gabbro = MOR_characterisation_serp_flux_FINAL.carbon_content_slow_ultraslow(samples,
                                                                                                            velocity)
                upper_volc_CO2 = lower_volc_CO2 = transition_CO2 = sheeted_dykes_CO2 = 0
                upper_volc_thickness = lower_volc_thickness = transition_thickness = sheeted_dykes_thickness = 0
                gabbros_thickness = thickness * volcanic_percent/100

                #incase a neg CO2 value comes back (i think this was fixed in previous tweaks, but
                #leaving it in to be sure)
                indices_neg_CO2_gabbro = CO2_gabbro < 0
                CO2_gabbro[indices_neg_CO2_gabbro] = 0
                CO2_volcanic = gabbros_thickness * CO2_gabbro

            else:
                carbon_max = 0
                tmp_DS = MOR_characterisation_serp_flux_FINAL.SR_and_dsSurf_inter_fast(samples,
                                                                                   velocity) #DS and thickness is called within this function

                thickness, upper_volc_thickness, lower_volc_thickness, transition_thickness, \
                sheeted_dykes_thickness, gabbros_thickness \
                = MOR_characterisation_serp_flux_FINAL.SR_and_thickness_inter_fast(samples,tmp_DS,volcanic_percent)

                upper_volc_CO2, lower_volc_CO2, transition_CO2, sheeted_dykes_CO2, \
                CO2_gabbro, bottom_water_temperature_multiplier = MOR_characterisation_serp_flux_FINAL.carbon_content_inter_fast(samples,
                                                                                            velocity,
                                                                                            bottom_water_temperature)
                #incase a neg CO2 value comes back (i think this was fixed in previous tweaks, but
                #leaving it in to be sure)
                indices_neg_CO2_gabbro = CO2_gabbro < 0
                CO2_gabbro[indices_neg_CO2_gabbro] = 0

                #gives us total CO2 storage in volcanics as a fraction (NOT A PERCENT)
                CO2_volcanic =  (upper_volc_thickness * upper_volc_CO2 + \
                    lower_volc_thickness * lower_volc_CO2 + \
                    transition_thickness * transition_CO2 + \
                    sheeted_dykes_thickness * sheeted_dykes_CO2 + \
                    gabbros_thickness * CO2_gabbro)/(thickness * volcanic_percent/100)

                #incase a neg CO2 value comes back (i think this was fixed in previous tweaks, but
                #leaving it in to be sure)
                indices_neg_CO2_volcanic = CO2_volcanic < 0
                CO2_volcanic[indices_neg_CO2_volcanic] = 0


            #carbon in serpentinite
            max_DS = 100
            #print tmp_DS
            if velocity[0] > 40:
                 #for fast ridges just assume max C is .32? questionable, no data available i think
                carbon_max = .32
            m = carbon_max/max_DS
            
            #bring it all together
            C_thickness_volcanics = thickness * volcanic_percent * 1/100 * CO2_volcanic * 1/100 * 12./44.
            C_thickness_serpentinites = thickness * per * 1/100 * 1000.0/865.0 * tmp_DS * m *1/100 * 12./44.
            C_total_volc_serp =  C_thickness_volcanics + C_thickness_serpentinites

            peridotites_point_thickness = thickness * per * 1/100
            serpentinites_point_thickness = peridotites_point_thickness * 1000.0/865.0 * tmp_DS *1/100

            #print PDF_parameters
            C_storage[time][SR].append(np.mean(velocity))
            C_storage[time][prop_per].append(np.mean(per))
            C_storage[time][DS].append(np.mean(tmp_DS))
            C_storage[time][CO2wt_mean].append((np.mean(upper_volc_CO2),
                                                 np.mean(lower_volc_CO2),
                                                 np.mean(transition_CO2),
                                                 np.mean(sheeted_dykes_CO2),
                                                 np.mean(CO2_gabbro)))
            C_storage[time][thick_mean].append((np.mean(thickness),
                                                np.mean(upper_volc_thickness),
                                                np.mean(lower_volc_thickness),
                                                np.mean(transition_thickness),
                                                np.mean(sheeted_dykes_thickness),
                                                np.mean(gabbros_thickness),
                                                np.mean(peridotites_point_thickness)))
            C_storage[time][vertical_area_mean].append((np.mean(upper_volc_thickness * velocity),
                                                        np.mean(lower_volc_thickness * velocity),
                                                        np.mean(transition_thickness * velocity),
                                                        np.mean(sheeted_dykes_thickness * velocity),
                                                        np.mean(gabbros_thickness * velocity),
                                                        np.mean(peridotites_point_thickness * velocity)))
            #multiply by mass C/mass CO2 to get C
            C_storage[time][C_total].append(np.mean(C_total_volc_serp))

            C_storage[time][C_total_vertical].append(np.mean(C_total_volc_serp)*velocity)

            C_storage[time][C_volcanic_serp].append((np.mean(C_thickness_volcanics),
                                                     np.mean(CO2_volcanic * 12./44.),
                                                     np.mean(C_thickness_serpentinites),
                                                     np.mean(tmp_DS * m)))

            C_storage[time][serpentinites_thick].append(np.mean(serpentinites_point_thickness))

            C_storage[time][serpentinites_vertical].append(np.mean(serpentinites_point_thickness) * velocity)

            C_storage[time][Index].append(index)
            C_storage[time][length].append(row.Length_km)
            C_storage[time][boundary].append(row.FeatureType)
            C_storage[time][StartLat].append(row.StartPointLat)
            C_storage[time][StartLon].append(row.StartPointLon)
            C_storage[time][EndLat].append(row.EndPointLat)
            C_storage[time][EndLon].append(row.EndPointLon)
            C_storage[time][RightPlateID].append(row.RightPlate)
            C_storage[time][LeftPlateID].append(row.LeftPlate)


    #save if wanted

    date = datetime.today().strftime('%Y-%m-%d')
    filename = 'C_storage_%s_plate_model_velocity.p' % date
    outfile = open('%s/%s' % (datadir, filename), 'wb')
    pickle.dump(C_storage, outfile)
    outfile.close()

    return(filename)


In [8]:
datadir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/Merdith_2019_Frontiers/Sample_Data/'
savedir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/Merdith_2019_Frontiers/output/'

#NB in cm/a
spread_dist = pickle.load(open('%sDistribution_for_random_spreading_Pacific100My.p.py' % datadir,'rb'))

# import previously made file in 'extract velocities'
file = '%sPlateBoundaryTypes_400-0_25Oct19.h5' % datadir
df = pd.read_hdf(file,'Statistics_table')
tags = df['FeatureType']

resolution = 1
samples=10000

In [9]:
#bottom water data
bottom_water_DF = pd.read_csv('%s/Bottom_water_temp_Muller_et_al.csv' % datadir)
#turn into arrays
x = np.asarray(bottom_water_DF['Age'])
y = np.asarray(bottom_water_DF['Temp'])
#flip arrays so they are going back in time
x = x[::-1]
y = y[::-1]
#interpolate into linear 1 Ma intervals
f = interpolate.interp1d(x, y)
x_new = np.linspace(0,400,401)
bottom_water_temperature_curve = f(x_new)

In [13]:
filename = crust_characterisation(0, 250, 1, df)

0 Ma
1 Ma
2 Ma
3 Ma
4 Ma
5 Ma
6 Ma
7 Ma
8 Ma
9 Ma
10 Ma
11 Ma
12 Ma
13 Ma
14 Ma
15 Ma
16 Ma
17 Ma
18 Ma
19 Ma
20 Ma
21 Ma
22 Ma
23 Ma
24 Ma
25 Ma
26 Ma
27 Ma
28 Ma
29 Ma
30 Ma
31 Ma
32 Ma
33 Ma
34 Ma
35 Ma
36 Ma
37 Ma
38 Ma
39 Ma
40 Ma
41 Ma
42 Ma
43 Ma
44 Ma
45 Ma
46 Ma
47 Ma
48 Ma
49 Ma
50 Ma
51 Ma
52 Ma
53 Ma
54 Ma
55 Ma
56 Ma
57 Ma
58 Ma
59 Ma
60 Ma
61 Ma
62 Ma
63 Ma
64 Ma
65 Ma
66 Ma
67 Ma
68 Ma
69 Ma
70 Ma
71 Ma
72 Ma
73 Ma
74 Ma
75 Ma
76 Ma
77 Ma
78 Ma
79 Ma
80 Ma
81 Ma
82 Ma
83 Ma
84 Ma
85 Ma
86 Ma
87 Ma
88 Ma
89 Ma
90 Ma
91 Ma
92 Ma
93 Ma
94 Ma
95 Ma
96 Ma
97 Ma
98 Ma
99 Ma
100 Ma
101 Ma
102 Ma
103 Ma
104 Ma
105 Ma
106 Ma
107 Ma
108 Ma
109 Ma
110 Ma
111 Ma
112 Ma
113 Ma
114 Ma
115 Ma
116 Ma
117 Ma
118 Ma
119 Ma
120 Ma
121 Ma
122 Ma
123 Ma
124 Ma
125 Ma
126 Ma
127 Ma
128 Ma
129 Ma
130 Ma
131 Ma
132 Ma
133 Ma
134 Ma
135 Ma
136 Ma
137 Ma
138 Ma
139 Ma
140 Ma
141 Ma
142 Ma
143 Ma
144 Ma
145 Ma
146 Ma
147 Ma
148 Ma
149 Ma
150 Ma
151 Ma
152 Ma
153 Ma
154 Ma
155 Ma
156 Ma
157 Ma
158 